# Get historical prices - OHLC

## Common imports and functions

In [4]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import datetime as dt
import calendar as cal
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", module="scipy", message="^internal gelsd")

from IPython.display import display
pd.options.display.max_columns = 50

## Read the categorized stocks

In [39]:
df_stocks = pd.read_csv('../dataset/KLSE_category_2.csv')
df_stocks.info()
df_stocks.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 4 columns):
code          108 non-null object
name          108 non-null object
yield_sum     108 non-null float64
yield_mean    108 non-null float64
dtypes: float64(2), object(2)
memory usage: 3.5+ KB


,code,name,yield_sum,yield_mean
0,5191,Tambun Indah Land Bhd (TAMB),49.69,16.5633
1,5272,Ranhill Holdings Ltd (RANH),48.11,24.0550
2,5161,JCY International Bhd (JCYI),47.34,23.6700
3,5254,Boustead Plantations Bhd (BOPL),47.29,23.6450
4,6084,Star Media Group Bhd (STAR),45.65,15.2167
5,5078,Marine & General Bhd (MARI),45.45,45.4500
6,8524,Taliworks Corporation Bhd (TWRK),42.03,21.0150
7,4502,Media Prima Bhd (MPRM),41.03,20.5150
8,5010,Tong Herr Resources Bhd (THRB),40.67,13.5567
9,3859,Magnum Bhd (MAGM),39.69,19.8450


## Get the URLs

In [75]:
df_lookup = pd.read_csv('../dataset/KLSE_lookup_investing.csv')
df_merged = pd.merge(df_stocks, df_lookup, how='left', left_on=['code'], right_on=['code'])
df_merged.rename(index=str, columns={'name_x': 'name'}, inplace=True)
df_merged.drop(['name_y'], axis=1,inplace=True)
df_merged['url'] = df_merged['url'].str[:-10]   ## Remove -dividends from the URL
# df_merged[~df_merged['name_y'].isnull()]
df_merged['url'].unique()
df_merged.to_csv('../dataset/KLSE_selected.csv'.format(c), encoding='utf-8', index=False, float_format='%g')
display(df_merged)

,code,name,yield_sum,yield_mean,url
0,5191,Tambun Indah Land Bhd (TAMB),49.69,16.56330,https://www.investing.com/equities/tambun-inda...
1,5272,Ranhill Holdings Ltd (RANH),48.11,24.05500,https://www.investing.com/equities/symphony-ho...
2,5161,JCY International Bhd (JCYI),47.34,23.67000,https://www.investing.com/equities/jcy-interna...
3,5254,Boustead Plantations Bhd (BOPL),47.29,23.64500,https://www.investing.com/equities/boustead-pl...
4,6084,Star Media Group Bhd (STAR),45.65,15.21670,https://www.investing.com/equities/star-media-...
5,5078,Marine & General Bhd (MARI),45.45,45.45000,https://www.investing.com/equities/silk-holdin...
6,8524,Taliworks Corporation Bhd (TWRK),42.03,21.01500,https://www.investing.com/equities/taliworks-c...
7,4502,Media Prima Bhd (MPRM),41.03,20.51500,https://www.investing.com/equities/media-prima...
8,5010,Tong Herr Resources Bhd (THRB),40.67,13.55670,https://www.investing.com/equities/tong-herr-r...
9,3859,Magnum Bhd (MAGM),39.69,19.84500,https://www.investing.com/equities/magnum-bhd
